## 基本爬蟲

In [11]:
# 導入庫
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from urllib.parse import quote
import time


# 自訂函數：搜尋並取回數據
def get_coupang_search_results(
        search_keyword: str,
        advanced_keywords: list[str]
):
    # 將搜尋關鍵字轉成網址安全格式，避免中文字或特殊符號導致網址錯誤
    encoded_keyword = quote(search_keyword)
    # 拼接查詢網址
    url = f"https://www.tw.coupang.com/search?q={encoded_keyword}&channel=user"
    # 建立 Chrome 的啟動選項設定物件
    options = Options()
    # 無頭模式
    options.add_argument("--headless=new")
    # 停用 GPU 加速，避免 headless 模式下 GPU 相容性問題
    options.add_argument("--disable-gpu")
    # 避免 Chrome 在沙箱環境下執行，有時對某些系統權限有問題
    options.add_argument("--no-sandbox")
    # 設定瀏覽器語言為繁體中文
    options.add_argument("--lang=zh-TW")
    # 建立 Chrome 瀏覽器實例
    # 透過 webdriver-manager 自動下載對應版本的 ChromeDriver
    driver = webdriver.Chrome(
        service=Service(
            ChromeDriverManager().install()),
            options=options
    )
    # 開啟指定的目標網址
    driver.get(url)
    # 等待一定時數
    time.sleep(5)

    html = driver.page_source
    # 若需重複操作瀏覽器可先保留不關閉
    driver.quit()

    soup = BeautifulSoup(html, "html.parser")
    product_cards = soup.select(
        "div.SearchResult_searchResultProduct___h6E9"
    )

    results = []
    matched_results = []

    for card in product_cards:
        try:
            # 完整內容文字
            full_text = card.get_text(separator=" ", strip=True)

            # 精簡標題（僅取 title 區塊）
            title_tag = card.select_one("div.Product_title__8K0xk")
            title = title_tag.get_text(strip=True) if title_tag else "N/A"

            # 價格
            price_tag = card.select_one(
                "span.Product_salePricePrice__2FbsL span"
            )
            price = price_tag.get_text(strip=True) if price_tag else "N/A"

            # 每單位價格
            unit_price_tag = card.select_one("div.Product_unitPrice__QQPdR")
            unit_price = unit_price_tag.get_text(strip=True) if unit_price_tag else "N/A"

            product = {
                "title": title,
                "full_text": full_text,
                "price": price,
                "unit_price": unit_price
            }

            results.append(product)

            # 進階條件：從完整描述中過濾
            if all(kw in full_text for kw in advanced_keywords):
                matched_results.append(product)

        except Exception as e:
            print("解析錯誤：", e)
            continue

    return results, matched_results

# 主程式區塊
if __name__ == "__main__":
    # 搜尋主關鍵字
    search_keyword = "BLUE BAY 倍力 Sense"

    # 進階篩選條件，需同時包含全部關鍵詞
    # advanced_keywords = []
    advanced_keywords = ["3種魚", "2袋"]

    # 執行爬取
    all_products, filtered_products = get_coupang_search_results(
        search_keyword,
        advanced_keywords
    )
    # 輸出關鍵字查看
    print(f"搜尋關鍵字：{search_keyword}")

    # 將所有結果輸出
    # print("所有搜尋結果：")
    # for idx, product in enumerate(all_products, 1):
    #     print(f"{idx}. 標題: {product['title']}")
    #     print(f"   價格: {product['price']}")
    #     print(f"   每單位: {product['unit_price']}")
    #     print(f"   完整內容: {product['full_text']}")
    #     print("-" * 60)

    # 輸出進階查詢結果
    if advanced_keywords:
        print(f"\n進階條件符合項目（包含：{'、'.join(advanced_keywords)}）：")
        for idx, product in enumerate(filtered_products, 1):
            print(f"{idx}. 標題: {product['title']}")
            print(f"   價格: {product['price']}")
            print(f"   每單位: {product['unit_price']}")
            print(f"   完整內容: {product['full_text']}")
            print("-" * 60)
    else:
        print("無提供進階篩選條件")

搜尋關鍵字：BLUE BAY 倍力 Sense

進階條件符合項目（包含：3種魚、2袋）：
1. 標題: BLUE BAY 倍力 Sense 全護低敏貓飼料 美膚爆毛配方 3種魚 + 台灣鱉蛋, 皮膚/毛髮, 1.5kg, 2袋
   價格: $765
   每單位: ($25.50/100g)
   完整內容: BLUE BAY 倍力 Sense 全護低敏貓飼料 美膚爆毛配方 3種魚 + 台灣鱉蛋, 皮膚/毛髮, 1.5kg, 2袋 特價 85折 $900 $765 ($25.50/100g) $135 優惠券 明天 5/17 (六) 預計送達 免運 ( 2 )
------------------------------------------------------------
